# LLM v2 Stock Analysis Notebook

This notebook demonstrates:
1. Stock data retrieval
2. LLM v2 analysis with detailed explanations
3. Display of evidence-based trading recommendations


In [1]:
# Setup and Imports
import os
import sys
import json
import asyncio
from pathlib import Path
from datetime import datetime
from dotenv import load_dotenv
import httpx
import pandas as pd

# Add project root to path
project_root = Path.cwd()
sys.path.insert(0, str(project_root))

# Load environment variables
env_path = project_root / ".env"
if env_path.exists():
    load_dotenv(env_path)
    print("✅ Environment variables loaded")
else:
    print("⚠️ .env file not found")

# Check API key
api_key = os.getenv("ANTHROPIC_API_KEY")
if api_key:
    print(f"✅ ANTHROPIC_API_KEY found (length: {len(api_key)})")
else:
    print("❌ ANTHROPIC_API_KEY not found in environment")

# API Configuration
API_BASE = "http://127.0.0.1:8000"
print(f"\n📡 API Base URL: {API_BASE}")


✅ Environment variables loaded
✅ ANTHROPIC_API_KEY found (length: 108)

📡 API Base URL: http://127.0.0.1:8000


## 1. Stock Data Retrieval


In [ ]:
# Import market data services
from services.marketdata.service import get_market_data_adapter
from services.calendar.service import get_earnings_calendar

def get_stock_data(ticker: str):
    """Retrieve comprehensive stock data"""
    print(f"\n📊 Fetching data for {ticker}...")
    
    market_data = get_market_data_adapter()
    calendar = get_earnings_calendar()
    
    data = {}
    
    try:
        # Get current quote
        quote = market_data.last_quote(ticker)
        data['price'] = quote.get('price', 0)
        data['spread'] = market_data.spread_proxy(ticker)
        print(f"  ✅ Price: ${data['price']:.2f}, Spread: ${data['spread']:.4f}")
    except Exception as e:
        print(f"  ❌ Error fetching quote: {e}")
        return None
    
    try:
        # Get historical data
        hist = market_data.daily_ohlc(ticker, lookback=365)
        if hist:
            data['historical'] = hist[-30:]  # Last 30 days
            print(f"  ✅ Historical data: {len(hist)} days")
        
        # Calculate metrics
        if len(hist) >= 10:
            recent_high = max([h['high'] for h in hist[-10:]])
            recent_low = min([h['low'] for h in hist[-10:]])
            price_position = (data['price'] - recent_low) / max(1e-6, recent_high - recent_low)
            
            avg_vol_30d = sum([h['volume'] for h in hist[-30:]]) / max(1, len(hist[-30:]))
            avg_vol_5d = sum([h['volume'] for h in hist[-5:]]) / max(1, len(hist[-5:]))
            volume_surge = avg_vol_5d / avg_vol_30d if avg_vol_30d > 0 else 1.0
            
            data['recent_high'] = recent_high
            data['recent_low'] = recent_low
            data['price_position'] = price_position
            data['volume_surge_ratio'] = volume_surge
            
            print(f"  ✅ Price position: {price_position:.2%}, Volume surge: {volume_surge:.2f}x")
    except Exception as e:
        print(f"  ⚠️ Error calculating metrics: {e}")
    
    try:
        # Get earnings calendar
        events = calendar.get_upcoming_events(ticker, days_ahead=30)
        if events:
            next_event = events[0]
            data['event_type'] = next_event.get('type', 'EARNINGS')
            data['days_to_event'] = next_event.get('days_until', 999)
            print(f"  ✅ Next event: {data['event_type']} in {data['days_to_event']} days")
        else:
            data['event_type'] = 'EARNINGS'
            data['days_to_event'] = 30
    except Exception as e:
        print(f"  ⚠️ Error fetching calendar: {e}")
        data['event_type'] = 'EARNINGS'
        data['days_to_event'] = 30
    
    return data

# Test with a ticker
ticker = "AAPL"
stock_data = get_stock_data(ticker)

if stock_data:
    print(f"\n📋 Stock Data Summary:")
    print(json.dumps({k: v for k, v in stock_data.items() if k != 'historical'}, indent=2))


## 2. LLM v2 Analysis


In [ ]:
async def analyze_with_llm_v2(ticker: str, stock_data: dict):
    """Call the LLM v2 API endpoint"""
    print(f"\n🤖 Calling LLM v2 API for {ticker}...")
    
    payload = {
        "ticker": ticker,
        "price": stock_data.get('price', 0),
        "spread": stock_data.get('spread', 0.01),
        "liquidity": stock_data.get('price', 0) * 1_000_000,  # Estimate
        "event_type": stock_data.get('event_type', 'EARNINGS'),
        "days_to_event": stock_data.get('days_to_event', 7),
        "expected_move": 4.0,
        "volume_surge_ratio": stock_data.get('volume_surge_ratio', 1.0),
        "recent_high": stock_data.get('recent_high', stock_data.get('price', 0)),
        "recent_low": stock_data.get('recent_low', stock_data.get('price', 0)),
        "price_position": stock_data.get('price_position', 0.5),
        "rolling_volatility_10d": 0.20,
        "context": [0.6, 0.7, 0.5, 0.8],
        "rank_components": {},
        "backtest_kpis": {},
        "decision_id": f"notebook_{ticker}_{int(datetime.now().timestamp())}"
    }
    
    try:
        async with httpx.AsyncClient(timeout=90.0) as client:
            response = await client.post(
                f"{API_BASE}/decision/propose",
                json=payload,
                headers={"Content-Type": "application/json"}
            )
            
            if response.status_code == 200:
                result = response.json()
                print("✅ LLM v2 analysis received")
                return result
            else:
                print(f"❌ API Error: {response.status_code}")
                print(response.text)
                return None
    except Exception as e:
        print(f"❌ Error calling API: {e}")
        return None

# Run analysis
if stock_data:
    result = await analyze_with_llm_v2(ticker, stock_data)
else:
    print("⚠️ No stock data available")
    result = None


## 3. Display Detailed Analysis


In [ ]:
def display_llm_analysis(result: dict):
    """Display LLM v2 analysis in a readable format"""
    if not result:
        print("❌ No analysis result to display")
        return
    
    llm_v2 = result.get('analysis', {}).get('llm_v2', {})
    
    if not llm_v2:
        print("⚠️ No LLM v2 data in response")
        print(f"\nFull response keys: {list(result.keys())}")
        return
    
    print("\n" + "="*80)
    print(f"📊 LLM v2 ANALYSIS FOR {result.get('analysis', {}).get('ticker', 'UNKNOWN').upper()}")
    print("="*80)
    
    # Core Verdicts
    print("\n🎯 VERDICTS:")
    print(f"  Intraday: {llm_v2.get('verdict_intraday', 'N/A')}")
    print(f"  Swing (1-5d): {llm_v2.get('verdict_swing_1to5d', 'N/A')}")
    print(f"  Confidence: {llm_v2.get('confidence', 0) * 100:.1f}%")
    
    # Detailed Rationale
    if llm_v2.get('plan', {}).get('rationale'):
        print("\n📝 DETAILED RATIONALE:")
        print("-"*80)
        rationale = llm_v2['plan']['rationale']
        # Wrap text for readability
        words = rationale.split()
        line = ""
        for word in words:
            if len(line + word) > 75:
                print(f"  {line}")
                line = word + " "
            else:
                line += word + " "
        if line:
            print(f"  {line}")
    
    # Key Explanations
    print("\n🔍 KEY EXPLANATIONS:")
    
    if llm_v2.get('room', {}).get('explain'):
        print("\n  📈 Room Analysis:")
        print(f"     {llm_v2['room']['explain']}")
    
    if llm_v2.get('participation', {}).get('explain'):
        print("\n  👥 Participation:")
        print(f"     {llm_v2['participation']['explain']}")
    
    if llm_v2.get('catalyst_alignment', {}).get('explain'):
        print("\n  ⚡ Catalyst Alignment:")
        print(f"     {llm_v2['catalyst_alignment']['explain']}")
    
    if llm_v2.get('statistical_analysis', {}).get('significance', {}).get('interpretation'):
        print("\n  📊 Statistical Significance:")
        print(f"     {llm_v2['statistical_analysis']['significance']['interpretation']}")
    
    # Trade Plan
    plan = llm_v2.get('plan', {})
    if plan.get('entry_price'):
        print("\n💼 TRADE PLAN:")
        print(f"  Entry: ${plan.get('entry_price', 0):.2f} ({plan.get('entry_type', 'N/A')})")
        print(f"  Stop: ${plan.get('stop_price', 0):.2f}")
        if plan.get('targets'):
            targets = ", ".join([f"${t:.2f}" for t in plan['targets']])
            print(f"  Targets: {targets}")
        print(f"  Timeout: {plan.get('timeout_days', 0)} days")
    
    # Risk Assessment
    risk = llm_v2.get('risk', {})
    if risk:
        print("\n⚠️ RISK ASSESSMENT:")
        print(f"  Policy Pass: {'✅' if risk.get('policy_pass') else '❌'}")
        if risk.get('warnings'):
            print("  Warnings:")
            for warning in risk['warnings']:
                print(f"    - {warning}")
    
    print("\n" + "="*80)

# Display the analysis
if result:
    display_llm_analysis(result)


## 4. Raw LLM Response (JSON)


In [ ]:
# Display full LLM v2 response as JSON
if result and result.get('analysis', {}).get('llm_v2'):
    llm_v2 = result['analysis']['llm_v2']
    print("\n📄 Full LLM v2 Response (JSON):")
    print("="*80)
    print(json.dumps(llm_v2, indent=2))
    
    # Show raw response if available
    if llm_v2.get('_raw_response'):
        print("\n\n📝 Raw LLM Response (from API):")
        print("="*80)
        print(llm_v2['_raw_response'])
else:
    print("⚠️ No LLM v2 data to display")


## 5. Quick Analysis Function


In [ ]:
async def quick_analysis(ticker: str):
    """Quick analysis function - does everything in one call"""
    print(f"\n{'='*80}")
    print(f"🚀 QUICK ANALYSIS: {ticker}")
    print(f"{'='*80}")
    
    # 1. Get stock data
    stock_data = get_stock_data(ticker)
    if not stock_data:
        print(f"❌ Failed to get stock data for {ticker}")
        return None
    
    # 2. Analyze with LLM
    result = await analyze_with_llm_v2(ticker, stock_data)
    if not result:
        print(f"❌ Failed to get LLM analysis for {ticker}")
        return None
    
    # 3. Display results
    display_llm_analysis(result)
    
    return result

# Run quick analysis - uncomment to use:
# result = await quick_analysis("AAPL")


## 6. Export Results


In [ ]:
def export_analysis(result: dict, filename: str = None):
    """Export analysis results to JSON file"""
    if not result:
        print("❌ No result to export")
        return
    
    if not filename:
        ticker = result.get('analysis', {}).get('ticker', 'UNKNOWN')
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"llm_analysis_{ticker}_{timestamp}.json"
    
    output_path = project_root / "output" / filename
    output_path.parent.mkdir(exist_ok=True)
    
    with open(output_path, 'w') as f:
        json.dump(result, f, indent=2, default=str)
    
    print(f"✅ Analysis exported to: {output_path}")
    return output_path

# Export current analysis
if result:
    export_analysis(result)
